## Import modules

In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

sns.set_theme()

## Joint train and test datasets available at UCI

In [88]:
col_names = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "target",
]
df_train = pd.read_csv("data/adult_data.csv", sep=",", header=0, names=col_names)
df_test = pd.read_csv("data/adult_data_test.csv", sep=",", header=0, names=col_names)
df = pd.concat([df_train, df_test], axis=0, ignore_index=True, verify_integrity=True)
df.to_csv("./data/us_census.csv", index=False)

## Read data and summarise each column

In [89]:
df = pd.read_csv("./data/us_census.csv", na_values=" ?")

In [90]:
## Let's see what values are in each column
for column in df.columns:
    print(df[column].value_counts())
    print()

36    1348
35    1337
33    1335
23    1329
31    1325
34    1303
28    1280
37    1280
30    1278
38    1264
32    1253
41    1235
27    1232
29    1223
24    1206
39    1205
25    1194
40    1187
22    1178
42    1165
26    1153
20    1113
43    1104
46    1097
21    1096
45    1096
47    1081
44    1067
19    1053
51     877
50     866
18     862
49     847
48     845
52     738
53     711
55     621
54     613
17     595
56     564
58     555
57     551
59     523
61     450
60     449
62     394
64     340
63     335
65     284
67     238
66     237
68     178
69     149
70     133
72     120
71     118
73     108
74      77
75      72
76      69
90      55
77      54
80      38
81      37
78      34
79      30
82      15
84      13
83      11
88       6
85       5
87       3
89       2
86       1
Name: age, dtype: int64

 Private             33905
 Self-emp-not-inc     3862
 Local-gov            3136
 State-gov            1980
 Self-emp-inc         1695
 Federal-gov          1432

In [91]:
## Check out the number of missing values
df.isna().sum(axis=0)

age                  0
workclass         2799
fnlwgt               0
education            0
education_num        0
marital_status       0
occupation        2809
relationship         0
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
native_country     857
target               0
dtype: int64

## Make some value replacements

In [92]:
## string type entries have a blank space in front of them
col_list = ["workclass", "education","marital_status",
            "occupation", "relationship", "race", "sex",
            "native_country", "target"]
df[col_list] = df[col_list].applymap(lambda string: string.strip() if string is not np.nan else string)

## Replace strings in target column by ones and zeroes
rep_dict = {
    "<=50K": 0,
    "<=50K.": 0,
    ">50K": 1,
    ">50K.": 1
}
df["target"].replace(to_replace=rep_dict, value=None, inplace=True)

## Drop columns which should not be used

In [94]:
## "education-num" is a categorical variable equivalent to
## "education".
##
## "fnlwgt" means "final-weight" and represents the number of
## people in the US population that each record represents, due
## to stratified sampling. Therefore, it should not be used when
## trying to predict individual income.

df.drop(labels=["education_num", "fnlwgt"], axis=1, inplace=True)

In [95]:
df.head()

,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,target
0,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
1,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
2,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
3,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
4,37,Private,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,0


In [96]:
df.corr()

,age,capital_gain,capital_loss,hours_per_week,target
age,1.000000,0.077227,0.056940,0.071558,0.230362
capital_gain,0.077227,1.000000,-0.031441,0.082157,0.223015
capital_loss,0.056940,-0.031441,1.000000,0.054467,0.147551
hours_per_week,0.071558,0.082157,0.054467,1.000000,0.227687
target,0.230362,0.223015,0.147551,0.227687,1.000000


In [38]:
for column in df.columns:
    print(df[column].value_counts())

36    1348
35    1337
33    1335
23    1329
31    1325
34    1303
28    1280
37    1280
30    1278
38    1264
32    1253
41    1235
27    1232
29    1223
24    1206
39    1205
25    1194
40    1187
22    1178
42    1165
26    1153
20    1113
43    1104
46    1097
21    1096
45    1096
47    1081
44    1067
19    1053
51     877
50     866
18     862
49     847
48     845
52     738
53     711
55     621
54     613
17     595
56     564
58     555
57     551
59     523
61     450
60     449
62     394
64     340
63     335
65     284
67     238
66     237
68     178
69     149
70     133
72     120
71     118
73     108
74      77
75      72
76      69
90      55
77      54
80      38
81      37
78      34
79      30
82      15
84      13
83      11
88       6
85       5
87       3
89       2
86       1
Name: age, dtype: int64
 Private             33905
 Self-emp-not-inc     3862
 Local-gov            3136
 ?                    2799
 State-gov            1980
 Self-emp-inc         1695


In [34]:
df["education-num"].value_counts()

9     15784
10    10878
13     8024
14     2657
11     2061
7      1811
12     1601
6      1389
4       955
15      834
5       756
8       657
16      594
3       509
2       247
1        83
Name: education-num, dtype: int64

In [36]:
df["marital-status"].value_counts()

 Married-civ-spouse       22379
 Never-married            16115
 Divorced                  6633
 Separated                 1530
 Widowed                   1518
 Married-spouse-absent      628
 Married-AF-spouse           37
Name: marital-status, dtype: int64

In [ ]:
df["ocupation"]

# --------------------------

In [4]:
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32560 entries, 0 to 32559
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32560 non-null  int64 
 1   workclass       32560 non-null  object
 2   fnlwgt          32560 non-null  int64 
 3   education       32560 non-null  object
 4   education-num   32560 non-null  int64 
 5   marital-status  32560 non-null  object
 6   occupation      32560 non-null  object
 7   relationship    32560 non-null  object
 8   race            32560 non-null  object
 9   sex             32560 non-null  object
 10  capital-gain    32560 non-null  int64 
 11  capital-loss    32560 non-null  int64 
 12  hours-per-week  32560 non-null  int64 
 13  native-country  32560 non-null  object
 14  target          32560 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16280 entries, 0 to 16279
Data columns (tot

In [5]:
df = pd.concat([df_train, df_test], axis=0, ignore_index=True, verify_integrity=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48840 entries, 0 to 48839
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48840 non-null  int64 
 1   workclass       48840 non-null  object
 2   fnlwgt          48840 non-null  int64 
 3   education       48840 non-null  object
 4   education-num   48840 non-null  int64 
 5   marital-status  48840 non-null  object
 6   occupation      48840 non-null  object
 7   relationship    48840 non-null  object
 8   race            48840 non-null  object
 9   sex             48840 non-null  object
 10  capital-gain    48840 non-null  int64 
 11  capital-loss    48840 non-null  int64 
 12  hours-per-week  48840 non-null  int64 
 13  native-country  48840 non-null  object
 14  target          48840 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [8]:
df.to_csv("./data/us_census.csv", index=False)